In [1]:
import torch
device = torch.device('cuda')

In [2]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('best_steering_model_xy_trt.pth')) # well trained road following model

model_trt_collision = TRTModule()
model_trt_collision.load_state_dict(torch.load('best_model_trt.pth')) # well trained collision avoidance model

<All keys matched successfully>

In [3]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [4]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetcam.utils import bgr8_to_jpeg
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224)

In [5]:
import ipywidgets

image_widget = ipywidgets.Image(format='jpeg', width=224, height=224)

camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

camera.running = True

In [6]:
from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

In [7]:
speed_control_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.001, description='speed control')

display(speed_control_slider)

blocked_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, orientation='horizontal', description='blocked')

display(image_widget)

display(ipywidgets.HBox([blocked_slider]))

FloatSlider(value=0.0, description='speed control', max=1.0, step=0.001)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [8]:
import math

count_stops = 0
go_on = 1
stop_time = 10 
car.throttle = speed_control_slider.value

def execute(change):
    global blocked_slider, robot, count_stops, stop_time, go_on, x, y, blocked_threshold
    global steering_gain, steering_bias
                
    steering_gain = 0.85
    steering_bias = 0.05
       
    image_preproc = preprocess(change['new']).to(device)
     
    
    
    prob_blocked = float(F.softmax(model_trt_collision(image_preproc), dim=1).flatten()[0])
    
    blocked_slider.value = prob_blocked    
    stop_time= 150
    
    if go_on == 1:    
        if prob_blocked > 0.9:
            count_stops += 1
            go_on = 2
        else:
            go_on = 1
            count_stops = 0
            car.throttle = speed_control_slider.value
            image = image_preproc.half()
            output = model_trt(image).detach().cpu().numpy().flatten()
            x = float(output[0])
            car.steering = x * steering_gain + steering_bias
    else:
        count_stops += 1
        if count_stops < stop_time:
            car.throttle = 0   
        else:
            go_on = 1
            count_stops = 0
            

execute({'new': camera.value})

In [9]:
camera.observe(execute, names='value')